## **NOTEBOOK 04-Experimentos 1/2/3 (prompting) y predicciones JSON estricto**
1. Cargar prompt_regression.jsonl (para depurar prompts rápido)
2. Definir los 3 prompts:
    - Exp1: solo nombres de etiquetas (sin criterios)
    - Exp2: criterios + memoria (tu memory_selected_FINAL)
    - Exp3: criterios + memoria + few-shot adicional
3. Generar predicciones JSON estricto (start/end/label) por documento
4. Guardar:
    - outputs/predictions/exp1.jsonl
    - outputs/predictions/exp2.jsonl
    - outputs/predictions/exp3.jsonl

A continuación, haremos:
- Cargar memoria final (memory_selected_FINAL.json) y el bloqueo anti-leakage.
- Ejecutar Experimento 1/2/3 (prompting) y guardar predicciones en JSONL.
- Validar salida con Pydantic (JSON estricto + offsets coherentes).
- Incluir una forma automática de seleccionar few-shot extra (Exp3) sin selección manual y sin leakage.

Nota operativa: NB04 asume que vamos a predecir sobre el gold (porque es el único con ground truth) y que NB05 evaluará contra gold_mvp. Los splits train/val/test/prompt_regression que hicimos en NB02 siguen siendo útiles para futuro fine-tuning y para testear prompts con texto no anotado, pero la evaluación real exige gold.

In [1]:
from pathlib import Path
ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists()
print("ROOT:", ROOT)

ROOT: /home/jovyan/inesagent


In [2]:
#comprobamos kernel y que user site es False
import site
site.ENABLE_USER_SITE


True

In [3]:
import os, site
print("PYTHONNOUSERSITE =", os.environ.get("PYTHONNOUSERSITE"))
print("ENABLE_USER_SITE =", site.ENABLE_USER_SITE)
print("USER_SITE =", site.getusersitepackages())


PYTHONNOUSERSITE = None
ENABLE_USER_SITE = True
USER_SITE = /home/jovyan/.local/lib/python3.11/site-packages


In [4]:
import sys

print("PY:", sys.executable)
assert "/inesagent_gpu/bin/python" in sys.executable, \
    "No estás en el kernel inesagent_gpu"

print("✅ Kernel correcto: inesagent_gpu")


PY: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
✅ Kernel correcto: inesagent_gpu


In [5]:
#celda bootstrap para que no se rompa nunca el environment 
from pathlib import Path
import sys, os

ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists(), f"ROOT no existe: {ROOT}"

# Asegura que el kernel es el correcto
print("Python:", sys.executable)
assert "/home/jovyan/.conda/envs/inesagent_gpu" in sys.executable, "Kernel incorrecto (no inesagent_gpu)"

# Cache HF persistente
os.environ["HF_HOME"] = "/home/jovyan/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/home/jovyan/.cache/huggingface/hub"
os.environ["HF_HUB_CACHE"] = "/home/jovyan/.cache/huggingface/hub"
os.environ["HF_DATASETS_CACHE"] = "/home/jovyan/.cache/huggingface/datasets"

print("ROOT OK:", ROOT)


Python: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
ROOT OK: /home/jovyan/inesagent


In [6]:
#Como cambiamos de local a server virtual, tenemos que cambiar los paths de todo 
from pathlib import Path
import os

# ROOT del proyecto en el servidor
ROOT = Path.home() / "inesagent"
assert ROOT.exists(), f"ROOT no existe: {ROOT}"

print("ROOT:", ROOT)


ROOT: /home/jovyan/inesagent


In [7]:
#comprobamos que el entorno es persistente (antes era efimero y se borraba al reiniciar server) 
import sys, os, pathlib
print(sys.executable)
print(pathlib.Path(sys.executable).exists())
print("HOME:", pathlib.Path.home())


/home/jovyan/.conda/envs/inesagent_gpu/bin/python
True
HOME: /home/jovyan


In [8]:
# Empezamos con el prompting. Imports y configuración de rutas (SERVER)

from pathlib import Path
import os, json, re, hashlib, random
from typing import List, Literal, Optional, Dict, Any

# ROOT del proyecto en el servidor
ROOT = Path.home() / "inesagent"
assert ROOT.exists(), f"ROOT no existe: {ROOT}"
print("ROOT:", ROOT)

# --- Rutas del proyecto ---
# Gold completo (si lo usas para resolver spans o construir índices)
# OJO: asegúrate del nombre real del archivo (jsonl vs json)
PATH_GOLD = ROOT / "gold" / "corpus_annotated.jsonl"   # cambia a .json si tu gold está en .json

# Memoria (elige el fichero que realmente tienes)
# Si tu fichero se llama memory_selected_CURATED.json, cámbialo aquí
PATH_MEMORY = ROOT / "outputs" / "memory" / "memory_selected_FINAL.json"

# Doc_uids bloqueados por leakage (si lo estás usando)
PATH_BLOCKED = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

# Splits (según tu carpeta)
SPLITS_DIR = ROOT / "outputs" / "splits"
PATH_VAL  = SPLITS_DIR / "gold_val.jsonl"
PATH_TEST = SPLITS_DIR / "gold_test.jsonl"
PATH_PR   = SPLITS_DIR / "prompt_regression.jsonl"

# Train: OJO con si es json o jsonl
PATH_TRAIN = SPLITS_DIR / "train.jsonl"

# Salida predicciones
OUT_PRED = ROOT / "outputs" / "predictions"
OUT_PRED.mkdir(parents=True, exist_ok=True)

# Seed reproducible
SEED = 42
random.seed(SEED)

# Etiquetas MVP
MVP_LABELS = ["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]

# --- Sanity checks (muy recomendables) ---
for p in [PATH_GOLD, PATH_MEMORY, PATH_VAL, PATH_TEST, PATH_PR]:
    print(str(p), "->", p.exists())

print(str(PATH_TRAIN), "->", PATH_TRAIN.exists(), "(train.jsonl puede ser opcional en NB04)")



ROOT: /home/jovyan/inesagent
/home/jovyan/inesagent/gold/corpus_annotated.jsonl -> True
/home/jovyan/inesagent/outputs/memory/memory_selected_FINAL.json -> True
/home/jovyan/inesagent/outputs/splits/gold_val.jsonl -> True
/home/jovyan/inesagent/outputs/splits/gold_test.jsonl -> True
/home/jovyan/inesagent/outputs/splits/prompt_regression.jsonl -> True
/home/jovyan/inesagent/outputs/splits/train.jsonl -> True (train.jsonl puede ser opcional en NB04)


In [9]:
# =========================
# NB04 — UTILS + PATHS
# =========================

from pathlib import Path
import os, json, re, hashlib, random
from typing import List, Dict, Any

# ------------------------------------------------------------------
# ROOT del proyecto (SERVER)
# ------------------------------------------------------------------
ROOT = Path.home() / "inesagent"
assert ROOT.exists(), f"ROOT no existe: {ROOT}"
print("ROOT:", ROOT)

# ------------------------------------------------------------------
# PATHS del proyecto
# ------------------------------------------------------------------
PATH_GOLD = ROOT / "gold" / "corpus_annotated.jsonl"

PATH_MEMORY = ROOT / "outputs" / "memory" / "memory_selected_FINAL.json"
PATH_BLOCKED = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

SPLITS_DIR = ROOT / "outputs" / "splits"
PATH_VAL  = SPLITS_DIR / "gold_val.jsonl"
PATH_TEST = SPLITS_DIR / "gold_test.jsonl"
PATH_PR   = SPLITS_DIR / "prompt_regression.jsonl"
PATH_TRAIN = SPLITS_DIR / "train.jsonl"  

OUT_PRED = ROOT / "outputs" / "predictions"
OUT_PRED.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# Configuración global
# ------------------------------------------------------------------
SEED = 42
random.seed(SEED)

MVP_LABELS = [
    "OBJETO",
    "PRECIO_DEL_CONTRATO",
    "DURACION_TOTAL_DEL_CONTRATO",
    "RESOLUCION",
]

# ------------------------------------------------------------------
# Funciones utilitarias
# ------------------------------------------------------------------
def is_jsonl(path: Path) -> bool:
    return path.suffix.lower() == ".jsonl"

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def stable_uid(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

# ------------------------------------------------------------------
# SYSTEM prompt (neutro, estable)
# ------------------------------------------------------------------
SYSTEM = """Eres un modelo de ANOTACIÓN de texto. Tu tarea es marcar spans (fragmentos) dentro de un texto.

Salida obligatoria: JSON estricto con:
- doc_uid: string
- spans: lista de objetos {tag, start, end}

Reglas:
- start/end son offsets de caracteres 0-based sobre el texto EXACTO.
- end es exclusivo.
- El substring text[start:end] debe existir exactamente.
- No inventes texto ni offsets.
- Puedes devolver múltiples spans.

Etiquetas permitidas (solo estas 4):
OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION

Devuelve SOLO JSON, sin explicaciones.
"""

# ------------------------------------------------------------------
# Sanity check de paths
# ------------------------------------------------------------------
for p in [PATH_GOLD, PATH_MEMORY, PATH_VAL, PATH_TEST, PATH_PR]:
    print(str(p), "->", p.exists())

print(str(PATH_TRAIN), "->", PATH_TRAIN.exists(), "(train puede ser opcional en NB04)")

print("UTILS cargadas correctamente")


ROOT: /home/jovyan/inesagent
/home/jovyan/inesagent/gold/corpus_annotated.jsonl -> True
/home/jovyan/inesagent/outputs/memory/memory_selected_FINAL.json -> True
/home/jovyan/inesagent/outputs/splits/gold_val.jsonl -> True
/home/jovyan/inesagent/outputs/splits/gold_test.jsonl -> True
/home/jovyan/inesagent/outputs/splits/prompt_regression.jsonl -> True
/home/jovyan/inesagent/outputs/splits/train.jsonl -> True (train puede ser opcional en NB04)
UTILS cargadas correctamente


In [10]:
#para asegurarnos de que sabemos de donde viene cualquier error (saldria "False") 
print("have tokenizer:", "tokenizer" in globals())
print("have model:", "model" in globals())
print("have run_llm:", "run_llm" in globals())
print("have SYSTEM:", "SYSTEM" in globals())


have tokenizer: False
have model: False
have run_llm: False
have SYSTEM: True


In [11]:
#Loaders (JSON / JSONL) + doc_uid estable
def load_jsonl(path: Path) -> List[dict]:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

gold_val = load_jsonl(PATH_VAL)
gold_test = load_jsonl(PATH_TEST)
prompt_regression = load_jsonl(PATH_PR)

print("gold_val:", len(gold_val))
print("gold_test:", len(gold_test))
print("prompt_regression:", len(prompt_regression))

# train.jsonl solo si lo necesitas
if PATH_TRAIN.exists():
    train = load_jsonl(PATH_TRAIN)
    print("train:", len(train) if isinstance(train, list) else type(train))



gold_val: 34
gold_test: 34
prompt_regression: 10
train: 279


In [12]:
#definimos is_jsonl (no es una funcion built-in)
from pathlib import Path
import json

def is_jsonl(path: Path) -> bool:
    return path.suffix.lower() == ".jsonl"

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


In [13]:
#Cargamos gold + memoria + blocked
# Gold
if not PATH_GOLD.exists():
    raise FileNotFoundError(f"No encuentro {PATH_GOLD}")
gold = load_json(PATH_GOLD) if not is_jsonl(PATH_GOLD) else load_jsonl(PATH_GOLD)

# Memoria final
if not PATH_MEMORY.exists():
    raise FileNotFoundError(f"No encuentro {PATH_MEMORY}")
memory_selected = load_json(PATH_MEMORY)

# Blocked uids por memoria (anti-leakage)
if not PATH_BLOCKED.exists():
    raise FileNotFoundError(f"No encuentro {PATH_BLOCKED}")
blocked = set(load_json(PATH_BLOCKED))

print("Gold docs:", len(gold))
print("Memoria ejemplos:", len(memory_selected))
print("Blocked doc_uids:", len(blocked))


Gold docs: 373
Memoria ejemplos: 8
Blocked doc_uids: 16


In [14]:
#definimos stable_uid para que pueda leer el resto de docs sin problema 
import hashlib

def stable_uid(text: str) -> str:
    """
    UID estable y determinista a partir del texto completo del documento.
    Usamos SHA1 para compatibilidad con el resto del pipeline.
    """
    if not isinstance(text, str):
        text = str(text)
    return hashlib.sha1(text.encode("utf-8")).hexdigest()


In [15]:
#Construimos gold_mvp y filtramos a 4 etiquetas 
def filter_tags_mvp(tags: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    out = []
    for t in tags:
        if t.get("tag") in MVP_LABELS:
            out.append(t)
    return out

gold_mvp = []
for d in gold:
    txt = d.get("text", "")
    if not txt:
        continue
    uid = stable_uid(txt)
    tags = filter_tags_mvp(d.get("tags", []))
    if not tags:
        continue
    gold_mvp.append({
        "doc_id": d.get("id"),
        "doc_uid": uid,
        "text": txt,
        "tags": tags
    })

print("Gold MVP docs:", len(gold_mvp))


Gold MVP docs: 373


In [16]:
#Split de evaluación dentro del gold (val/test) sin leakage
#Esto crea gold_val y gold_test evitando documentos bloqueados (usados en memoria/few-shot)

eval_pool = [d for d in gold_mvp if d["doc_uid"] not in blocked]
random.shuffle(eval_pool)

test_n = max(30, int(0.10 * len(eval_pool)))   # mínimo 30 si hay suficiente
val_n  = max(30, int(0.10 * len(eval_pool)))

gold_test = eval_pool[:test_n]
gold_val  = eval_pool[test_n:test_n+val_n]
gold_train_pool = eval_pool[test_n+val_n:]  # pool para few-shot extra, etc.

print("Eval pool:", len(eval_pool))
print("gold_val:", len(gold_val), "gold_test:", len(gold_test), "gold_train_pool:", len(gold_train_pool))


Eval pool: 357
gold_val: 35 gold_test: 35 gold_train_pool: 287


In [17]:
#instalamos pydantic, que usaremos en la siguiente celda 
import sys
!{sys.executable} -m pip install -U "pydantic>=2"



In [18]:
#comprobamos que está
import sys, site
print("PYTHON =", sys.executable)
print("site-packages =", site.getsitepackages())
print("user-site =", site.getusersitepackages())

import pydantic
print("pydantic =", pydantic.__version__, pydantic.__file__)



PYTHON = /home/jovyan/.conda/envs/inesagent_gpu/bin/python
site-packages = ['/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages']
user-site = /home/jovyan/.local/lib/python3.11/site-packages
pydantic = 2.12.5 /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/pydantic/__init__.py


In [19]:
#Pydantic: esquema de salida estricto (spans)

from typing import List, Optional, Literal, Dict, Union, Annotated, Any

from pydantic import BaseModel, Field, ValidationError, field_validator

Label = Literal["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]

class SpanPred(BaseModel):
    tag: Label
    start: int = Field(ge=0)
    end: int = Field(ge=0)

    @field_validator("end")
    @classmethod
    def end_must_be_gt_start(cls, v, info):
        start = info.data.get("start")
        if start is not None and v <= start:
            raise ValueError("end must be > start")
        return v

class DocPred(BaseModel):
    doc_uid: str
    spans: List[SpanPred]

def validate_pred(doc_uid: str, text: str, pred_json: Dict[str, Any]) -> DocPred:
    obj = DocPred(**pred_json)
    if obj.doc_uid != doc_uid:
        raise ValueError("doc_uid mismatch")

    # offsets in range + span not empty
    for sp in obj.spans:
        if sp.end > len(text):
            raise ValueError("span end out of range")
        if not text[sp.start:sp.end].strip():
            raise ValueError("empty span text")
    return obj


In [20]:
#Render de memoria (para Exp2/Exp3)

# Índice para recuperar texto por doc_uid (solo de gold)
uid_to_text = {d["doc_uid"]: d["text"] for d in gold_mvp}

def render_memory_blocks(memory: List[Dict[str, Any]]) -> str:
    blocks = []
    for ex in memory:
        uid = ex["doc_uid"]
        txt = uid_to_text.get(uid, "")
        s, e = ex["start"], ex["end"]
        span_txt = txt[s:e].replace("\n", " ").strip()
        blocks.append(
            f"- LABEL: {ex['label']}\n"
            f"  CRITERION: {ex['criterion']}\n"
            f"  EXAMPLE_SPAN: {span_txt}"
        )
    return "\n".join(blocks)

memory_text = render_memory_blocks(memory_selected)
print(memory_text[:1200], "...\n")


- LABEL: DURACION_TOTAL_DEL_CONTRATO
  CRITERION: P4 – criterio único (subtipos solo ejemplificativos)
  EXAMPLE_SPAN: El presente contrato tendrá un plazo de vigencia desde el 1 de enero de 2024 (o desde la fecha de formalización del contrato si fuera posterior a ésta) y hasta el 31 de diciembre de 2024.
- LABEL: DURACION_TOTAL_DEL_CONTRATO
  CRITERION: P4 – criterio único (subtipos solo ejemplificativos)
  EXAMPLE_SPAN: DURACIÓN DEL CONTRATO. El Contrato tendrá una duración de VEINTICUATRO (24) MESES, contados a partir del día 1 de enero de 2024.
- LABEL: DURACION_TOTAL_DEL_CONTRATO
  CRITERION: P4 – criterio único (subtipos solo ejemplificativos)
  EXAMPLE_SPAN: El plazo de vigencia es de 2 años desde la formalización del contrato, no pudiéndose prorrogar.
- LABEL: DURACION_TOTAL_DEL_CONTRATO
  CRITERION: P4 – criterio único (subtipos solo ejemplificativos)
  EXAMPLE_SPAN: El plazo de prestación del servicio será de 12 meses desde el 01/03/2024 hasta 28/02/2025 de acuerdo con el art

**Para la selección automática de few-shot extra (Exp3) de forma no manual, debemos saber:**

Política (automática, reproducible):
- Seleccionar ejemplos desde gold_train_pool (no bloqueados, no val/test).
- Evitar doc_uids ya usados en memoria.
- Para P1/P2: 1 ejemplo extra por etiqueta (diversidad lexical simple).
- Para P4: 2 ejemplos extra (uno con “formalización”, otro con fechas).
- Para P9: 2 ejemplos extra (uno con “LCSP/artículos”, otro con “incumplimiento/causas”).

In [21]:
#Selección automática de few-shot extra (Exp3) (no manual)
memory_uids = {ex["doc_uid"] for ex in memory_selected}

def spans_for_label(doc, label):
    txt = doc["text"]
    for t in doc["tags"]:
        if t["tag"] == label:
            s, e = int(t["start"]), int(t["end"])
            yield (s, e, txt[s:e])

def pick_first_match(label: str, regex: re.Pattern, k: int) -> List[Dict[str, Any]]:
    picked = []
    used = set()
    for d in gold_train_pool:
        if d["doc_uid"] in memory_uids:
            continue
        if d["doc_uid"] in used:
            continue
        for s, e, span_txt in spans_for_label(d, label):
            if regex.search(span_txt):
                picked.append({
                    "label": label,
                    "criterion": f"FEWSHOT_EXTRA ({label})",
                    "doc_uid": d["doc_uid"],
                    "start": s,
                    "end": e,
                })
                used.add(d["doc_uid"])
                break
        if len(picked) >= k:
            break
    return picked

fewshot_extra = []
# OBJETO: 1 extra (p.ej. contiene "SERVICIO" / "SUMINISTRO" típicos)
fewshot_extra += pick_first_match("OBJETO", re.compile(r"\bSERVICIO\b|\bSUMINISTRO\b|\bOBRAS\b", re.IGNORECASE), k=1)
# PRECIO: 1 extra (IVA / euros)
fewshot_extra += pick_first_match("PRECIO_DEL_CONTRATO", re.compile(r"€|euros|IVA", re.IGNORECASE), k=1)
# DURACION: 2 extras (formalización + fechas)
fewshot_extra += pick_first_match("DURACION_TOTAL_DEL_CONTRATO", re.compile(r"formalizaci[oó]n", re.IGNORECASE), k=1)
fewshot_extra += pick_first_match("DURACION_TOTAL_DEL_CONTRATO", re.compile(r"\b\d{1,2}\s+de\s+[a-záéíóú]+\s+de\s+\d{4}\b", re.IGNORECASE), k=1)
# RESOLUCION: 2 extras (LCSP/artículos + incumplimiento/causas)
fewshot_extra += pick_first_match("RESOLUCION", re.compile(r"\bLCSP\b|Ley\s+9/2017|art[ií]cul", re.IGNORECASE), k=1)
fewshot_extra += pick_first_match("RESOLUCION", re.compile(r"incumplim|causas?\s+de\s+resoluci", re.IGNORECASE), k=1)

print("fewshot_extra:", len(fewshot_extra))
for ex in fewshot_extra:
    txt = uid_to_text[ex["doc_uid"]]
    print(ex["label"], ex["doc_uid"][:10], txt[ex["start"]:ex["end"]][:140].replace("\n"," "))


fewshot_extra: 6
OBJETO 5ff3583e80 SUMINISTRO DE DOS FURGONES (UN VEHÍCULO DE COMBUSTIÓN ASISTENCIAL CARROZADO COMO AMBULANCIA PARA EL SERVICIO DE PROTECCIÓN CIVIL Y UN FURGÓN
PRECIO_DEL_CONTRATO 5ff3583e80 128.491,47 euros, IVA incluido
DURACION_TOTAL_DEL_CONTRATO 98bf9c3389 El plazo de duración del contrato es de 24 meses. El objeto del contrato consta de 2 revisiones, la primera se realizará, tras la formalizac
DURACION_TOTAL_DEL_CONTRATO 6bb61ba065 El plazo de ejecución de la prestación objeto de este contrato será desde el 01 de diciembre de 2023, o desde la firma del contrato si ésta 
RESOLUCION 98bf9c3389 Serán causas de resolución de este contrato las establecidas en los artículos 211 y 313 de la LCSP.
RESOLUCION 98bf9c3389 Serán causas de resolución de este contrato las establecidas en los artículos 211 y 313 de la LCSP.


**Guardado opcional**

Cómo se usará en Exp3: memory_selected + fewshot_extra (y se bloquean también sus doc_uids si luego queremos máxima limpieza).

In [22]:
#Guardado opcional
(PATH_GOLD.parent.parent / "outputs" / "memory" / "fewshot_extra.json").write_text(
    json.dumps(fewshot_extra, ensure_ascii=False, indent=2), encoding="utf-8"
)


1139

In [23]:
#Templates de prompts (Exp1/Exp2/Exp3)
SYSTEM = (
    "Eres un anotador jurídico experto en contratos del sector público en España. "
    "Tu tarea es identificar fragmentos textuales (segmentos/oraciones/cláusulas) que cumplen una función jurídica "
    "y devolver sus offsets start/end con una etiqueta. "
    "No extraigas palabras sueltas: selecciona el fragmento mínimo suficiente que expresa la función."
)

LABEL_DEFS_MIN = (
    "Etiquetas disponibles:\n"
    "- OBJETO: prestación principal del contrato.\n"
    "- PRECIO_DEL_CONTRATO: cuantía del contrato (con o sin IVA/IGIC según corresponda).\n"
    "- DURACION_TOTAL_DEL_CONTRATO: vigencia/plazo total y condiciones de inicio del plazo.\n"
    "- RESOLUCION: causas y régimen de resolución/extinción del contrato.\n"
)

OUTPUT_CONSTRAINTS = (
    "Devuelve SOLO un JSON válido con esta forma:\n"
    "{\n"
    '  "doc_uid": "<doc_uid>",\n'
    '  "spans": [ {"tag":"OBJETO","start":0,"end":10}, ... ]\n'
    "}\n"
    "Reglas:\n"
    "- start/end son offsets de caracteres sobre el texto original.\n"
    "- end es exclusivo.\n"
    "- No inventes offsets.\n"
    "- No incluyas texto fuera del JSON.\n"
)

def make_user_prompt_exp1(doc_uid: str, text: str) -> str:
    return (
        LABEL_DEFS_MIN + "\n"
        + OUTPUT_CONSTRAINTS + "\n"
        + f'DOC_UID: {doc_uid}\n'
        + "TEXTO:\n"
        + text
    )

def make_user_prompt_exp2(doc_uid: str, text: str, memory_str: str) -> str:
    return (
        LABEL_DEFS_MIN + "\n"
        + "Criterios y ejemplos (memoria):\n"
        + memory_str + "\n\n"
        + OUTPUT_CONSTRAINTS + "\n"
        + f'DOC_UID: {doc_uid}\n'
        + "TEXTO:\n"
        + text
    )

def make_user_prompt_exp3(doc_uid: str, text: str, memory_str: str, fewshot_str: str) -> str:
    return (
        LABEL_DEFS_MIN + "\n"
        + "Criterios y ejemplos (memoria):\n"
        + memory_str + "\n\n"
        + "Few-shot adicional:\n"
        + fewshot_str + "\n\n"
        + OUTPUT_CONSTRAINTS + "\n"
        + f'DOC_UID: {doc_uid}\n'
        + "TEXTO:\n"
        + text
    )


## Cargamos el modelo (GPU)

Para nuestra instancia con vGPU 20GB (INESData), lo más práctico es Transformers + 4-bit. vLLM va muy bien, pero requiere más setup; para un NB, Transformers suele ser más directo.

Elegimos entre: **Mistral 7B Instruct** o **Llama 3.1 8B Instruct**.

Modelo recomendado para tus experimentos (España, jurídico, español)

Para prompting en español con 20GB VRAM:
- Meta-Llama-3.1-8B-Instruct en 4-bit. Suele ser más robusto “cero/few-shot” y consistente en instrucciones largas (opción recomendada)
- Mistral-7B-Instruct-v0.3 en 4-bit. Muy eficiente y suele rendir bien con prompts estructurados (alternativa muy válida)

Qué elegir ahora: yo elegiría Llama 3.1 8B Instruct si tienes acceso al checkpoint. Si no, usa Mistral 7B Instruct v0.3.

Por ahora, elegimos **Modelo: Meta-Llama-3.1-8B-Instruct (4-bit)**

Motivo: mejor comportamiento que Mistral 7B con:
- instrucciones largas,
- criterios explícitos,
- memoria estructurada,
- español jurídico.

**!! Como experimento, probaremos los 2 a ver cuál da mejores resultados**
Solo habría que cambiar:
- *MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"*
- por *MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"*

## Instalación transformers y modelo

In [24]:
import sys, site, os, subprocess, textwrap

print("PYTHON:", sys.executable)
print("USER_SITE:", site.getusersitepackages())
print("PATH has ~/.local/bin?:", os.path.expanduser("~/.local/bin") in os.environ.get("PATH",""))

def pip_show(pkg):
    r = subprocess.run([sys.executable, "-m", "pip", "show", pkg], capture_output=True, text=True)
    return r.stdout.strip() if r.returncode == 0 else "(not installed)"

for p in ["transformers","huggingface-hub","accelerate","bitsandbytes","torch"]:
    print("\n===", p, "===\n", pip_show(p))


PYTHON: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
USER_SITE: /home/jovyan/.local/lib/python3.11/site-packages
PATH has ~/.local/bin?: False

=== transformers ===
 Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by:

=== huggingface-hub ===
 Name: huggingface_hub
Version: 0.36.1
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugg

In [25]:
#boostrap estable, instala/actualiza todo en el py del kernel 
import sys, subprocess

pkgs = [
    # núcleo HF (versiones compatibles)
    "huggingface_hub>=0.30,<1.0",
    "transformers>=4.45,<4.47",
    "accelerate>=0.34,<2.0",
    "tokenizers>=0.20,<0.22",
    "safetensors>=0.4",

    # utilidades frecuentes
    "sentencepiece",
    "protobuf<5",  # evita roturas típicas con algunas wheels
    "jsonschema",
    "pydantic>=2.5",
    "tqdm",
    "regex",
    "numpy",

    # cuantización 4-bit (si tu runtime lo soporta)
    "bitsandbytes>=0.43",
]

cmd = [sys.executable, "-m", "pip", "install", "--user", "-U"] + pkgs
print("RUN:", " ".join(cmd))
subprocess.check_call(cmd)

print("\n✅ Instalación/upgrade terminado.")
print("👉 IMPORTANTE: Reinicia el kernel AHORA (Kernel -> Restart) y vuelve a ejecutar las celdas.")


RUN: /home/jovyan/.conda/envs/inesagent_gpu/bin/python -m pip install --user -U huggingface_hub>=0.30,<1.0 transformers>=4.45,<4.47 accelerate>=0.34,<2.0 tokenizers>=0.20,<0.22 safetensors>=0.4 sentencepiece protobuf<5 jsonschema pydantic>=2.5 tqdm regex numpy bitsandbytes>=0.43
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 77.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/7 [sympy]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [jsonschema]7 [jsonschema]specifications]


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



✅ Instalación/upgrade terminado.
👉 IMPORTANTE: Reinicia el kernel AHORA (Kernel -> Restart) y vuelve a ejecutar las celdas.


In [26]:
#verificacion despues de reiniciar el kernel 
import transformers, huggingface_hub, accelerate
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("accelerate:", accelerate.__version__)

from transformers import BitsAndBytesConfig
print("✅ BitsAndBytesConfig OK")


/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


transformers: 4.46.3
huggingface_hub: 0.36.1
accelerate: 1.12.0
✅ BitsAndBytesConfig OK


In [27]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.6/915.6 MB 88.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 127.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 240.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 316.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 86.3 MB/s eta 0:00:00 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 66.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.6 MB

In [28]:
#comprobamos que los paquetes se estén instalando en inesagent_gpu 
import accelerate
print("accelerate:", accelerate.__version__, accelerate.__file__)


accelerate: 1.12.0 /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/accelerate/__init__.py


In [29]:
!pip install -U bitsandbytes 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 92.3 MB/s eta 0:00:0000:0100:01


In [30]:
!pip install -U sentencepiece 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.0 MB/s eta 0:00:00a 0:00:01


In [31]:
#Instalamos bitsandbites (libreria para optimizar modelos de IA) (solo 1 vez) y reiniciamos Kernel
#separo los install para que no pete (con !pip install -U transformers accelerate bitsandbytes sentencepiece de golpe se muere)
!pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.9 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.5 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [32]:
#Verifico versiones
import transformers, huggingface_hub, accelerate
import bitsandbytes as bnb

print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("accelerate:", accelerate.__version__)
print("bitsandbytes:", bnb.__version__)


transformers: 4.46.3
huggingface_hub: 0.36.1
accelerate: 1.12.0
bitsandbytes: 0.49.1


In [33]:
#reparo transformers (pip) y reinicio kernel para evitar errores "Could not find LlamaForCausalLM" (mezcla de paquetes)
#reduzco transformers a 4.45.2 (más estable para Llama 3)
!pip install -U --no-cache-dir --force-reinstall \
  "transformers==4.45.2" \
  "accelerate>=0.33" \
  "huggingface_hub>=0.34,<1.0" \
  "safetensors>=0.4" \
  "sentencepiece" \
  "bitsandbytes"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 172.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 113.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 78.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 91.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 267.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 361.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.0 MB/s eta 0:00:00a 0:00:

In [34]:
#despues de reinstalar torch CUDA con mamba (piso el torch de pip por tener otra version).  
#despues en terminal fuerzo a eliminar el torch de pip
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)


2.5.1
True
12.1


In [35]:
#Verifico versiones
import transformers, huggingface_hub, accelerate
import bitsandbytes as bnb

print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("accelerate:", accelerate.__version__)
print("bitsandbytes:", bnb.__version__)


transformers: 4.46.3
huggingface_hub: 0.36.1
accelerate: 1.12.0
bitsandbytes: 0.49.1


In [36]:
#después de instalar pytorch y el resto de paquetes, descomprimir el zip de carpetas de inesagent verificamos que esté todo:
from pathlib import Path
ROOT = Path.home() / "inesagent"
assert ROOT.exists(), ROOT
print("ROOT OK:", ROOT)

print("config:", (ROOT/"config").exists())
print("gold:", (ROOT/"gold").exists())
print("outputs:", (ROOT/"outputs").exists())


ROOT OK: /home/jovyan/inesagent
config: True
gold: True
outputs: True


In [37]:
#sanitycheck GPU y VRAM 
import torch
print("cuda:", torch.cuda.is_available())
print("gpu:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    print("VRAM (GB):", round(props.total_memory/1e9, 2))


cuda: True
gpu: NVIDIA A100-PCIE-40GB MIG 7g.40gb
VRAM (GB): 42.29


In [38]:
#aseguramos dependencias 
import sys
!{sys.executable} -m pip install -U --no-cache-dir \
  "transformers==4.45.2" "accelerate>=0.33" "bitsandbytes" \
  "huggingface_hub>=0.34,<1.0" "safetensors" "sentencepiece" "requests" "tqdm"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 65.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 49.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [sympy]  WARNING: The script isympy is installed in '/home/jovyan/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]  WARNING: The script transformers-cli is installed in '/home/jovyan/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]


In [51]:
#Hacemos login Huggingface con nuestro token (inesagent-token) 
from huggingface_hub import login, hf_hub_download

login(token="hf_REDACTED")  #mi token

# test gated
hf_hub_download("meta-llama/Meta-Llama-3.1-8B-Instruct", "config.json")
print("HF login + acceso OK")


HF login + acceso OK


In [52]:
#limpiamos VRAM antes de cargar para evitar fragmentación
import torch, gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
print("GPU cache cleared")


GPU cache cleared


## Cargamos el modelo 
- MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"
- MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [53]:
#comprobar que estamos usando transformers desde env/kernal, y no user-site. Limpio local y reinstalo dentro del env 
import sys, transformers
print("PY:", sys.executable)
print("transformers:", transformers.__version__)
print("transformers file:", transformers.__file__)


PY: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
transformers: 4.46.3
transformers file: /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/transformers/__init__.py


In [54]:
import sys, site
print("PY:", sys.executable)
print("ENABLE_USER_SITE:", site.ENABLE_USER_SITE)
print("USER_SITE:", site.getusersitepackages())
print("sys.path (primeros 8):")
for p in sys.path[:8]:
    print(" -", p)


PY: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
ENABLE_USER_SITE: True
USER_SITE: /home/jovyan/.local/lib/python3.11/site-packages
sys.path (primeros 8):
 - /home/jovyan/.conda/envs/inesagent_gpu/lib/python311.zip
 - /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11
 - /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/lib-dynload
 - 
 - /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages
 - /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/setuptools/_vendor
 - /tmp/tmpysprx0hf


In [55]:
#Desactivar user-site dentro del kernel (parche inmediato)
import sys, site, os

user_site = site.getusersitepackages()
print("USER_SITE:", user_site)

# 1) quita user-site del sys.path si está
sys.path = [p for p in sys.path if p != user_site]

# 2) y además fija la variable para procesos hijos (pip, etc.)
os.environ["PYTHONNOUSERSITE"] = "1"

print("OK: user-site fuera de sys.path y PYTHONNOUSERSITE=1")


USER_SITE: /home/jovyan/.local/lib/python3.11/site-packages
OK: user-site fuera de sys.path y PYTHONNOUSERSITE=1


In [56]:
#“Purge” de los paquetes en ~/.local (para que no vuelvan) 
import sys, os
os.environ["PYTHONNOUSERSITE"] = "1"

# Desinstala de user-site (si existían ahí)
!python -m pip uninstall -y transformers tokenizers huggingface-hub safetensors accelerate bitsandbytes peft sentencepiece

# Limpia caches (opcional pero recomendable)
!rm -rf /home/jovyan/.cache/huggingface /home/jovyan/.cache/torch /home/jovyan/.cache/pip


In [57]:
#veriricamos que seguimos dentro del entorno
import sys
print("PYTHON:", sys.executable)


PYTHON: /home/jovyan/.conda/envs/inesagent_gpu/bin/python


In [58]:
#instalamos dependencias dentro del env (correcto y seguro) 
!PIP_USER=false PYTHONNOUSERSITE=1 {sys.executable} -m pip install --no-user --no-warn-script-location \
    -U pip setuptools wheel packaging

!PIP_USER=false PYTHONNOUSERSITE=1 {sys.executable} -m pip install --no-user --no-warn-script-location \
    -U "huggingface_hub>=0.30,<1.0" \
       "transformers>=4.46,<4.47" \
       tokenizers safetensors accelerate bitsandbytes sentencepiece


In [59]:
#despues de instalar, ejecutamos para comprobar que ya no hay nada en .local
import transformers, sys
print("Transformers version:", transformers.__version__)
print("Transformers path:", transformers.__file__)


Transformers version: 4.46.3
Transformers path: /home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/transformers/__init__.py


In [60]:
#sigue saliendo .local, por lon que vamos a borrar manualmente todo lo instalado ahi para que transformers no se aloje ahi
import shutil, site, os
user_site = site.USER_SITE
print("User-site:", user_site)

if os.path.exists(user_site):
    shutil.rmtree(user_site)
    print("User-site deleted.")
else:
    print("User-site not found.")


User-site: /home/jovyan/.local/lib/python3.11/site-packages
User-site not found.


In [63]:
#Hacemos login Huggingface con nuestro token (inesagent-token) 
from huggingface_hub import login, hf_hub_download

login(token="hf_REDACTED")  #mi token

# test gated
hf_hub_download("meta-llama/Meta-Llama-3.1-8B-Instruct", "config.json")
print("HF login + acceso OK")


HF login + acceso OK


In [66]:
#checks rapidos
import torch, transformers
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))


torch: 2.5.1
transformers: 4.46.3
cuda available: True
gpu: NVIDIA A100-PCIE-40GB MIG 7g.40gb


In [67]:
#pipeline en GPU + control de longitud (lo importante)
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.1-8B-Instruct",
    device_map="auto",                      # <-- usa GPU automáticamente
    model_kwargs={"torch_dtype": torch.float16},  # fp16 suele ir bien en vGPU
)

messages = [
    {"role": "user", "content": "Who are you?"},
]

out = pipe(
    messages,
    max_new_tokens=64,          # <-- evita el error de max_length
    do_sample=False,            # determinista (útil para anotación/IE)
    return_full_text=False,     # solo respuesta (opcional)
)

print(out[0]["generated_text"])



Loading checkpoint shards: 100%|██████████| 4/4 [01:56<00:00, 29.11s/it]
/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


## Los modelos “Instruct” suelen esperar un formato de chat (plantilla)

pipeline puede manejar messages, pero si quieres control total (y reproducibilidad), usa `apply_chat_template`

In [68]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

out = pipe(
    prompt,
    max_new_tokens=64,
    do_sample=False,
    return_full_text=False,
)

print(out[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [61]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

use_cuda = torch.cuda.is_available()
print("CUDA:", use_cuda, "| device:", torch.cuda.get_device_name(0) if use_cuda else "CPU")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# Intento 4-bit (bitsandbytes)
model = None
if use_cuda:
    try:
        from transformers import BitsAndBytesConfig

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map={"": 0},  # fuerza GPU:0
        )
        print("✅ Modelo cargado en 4-bit (GPU)")

    except Exception as e:
        print("⚠️ No se pudo cargar 4-bit (bitsandbytes). Motivo:", repr(e))
        print("➡️ Fallback a FP16 en GPU...")

if model is None:
    # Fallback: FP16 (GPU) o FP32 (CPU)
    dtype = torch.float16 if use_cuda else torch.float32
    device_map = {"": 0} if use_cuda else None

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=dtype,
        device_map=device_map,
    )
    print("✅ Modelo cargado en", "FP16 GPU" if use_cuda else "FP32 CPU")

model.eval()


CUDA: True | device: NVIDIA A100-PCIE-40GB MIG 7g.40gb


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-6981e223-493d71bc63ff01003a384538;33ee3625-22c0-4593-968a-cf372731f65e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import os, json, re, hashlib
from pathlib import Path
from typing import List, Dict, Any, Optional

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig


In [ ]:
# --- Paths del proyecto ---
ROOT = Path("/home/jovyan/inesagent")

PATH_INPUT_JSONL = ROOT / "outputs" / "splits" / "gold_test.jsonl"   # o prompt_regression.jsonl
PATH_MEMORY = ROOT / "outputs" / "memory" / "memory_selected_CURATED.json"

OUT_DIR = ROOT / "outputs" / "exp4"
OUT_DIR.mkdir(parents=True, exist_ok=True)
PATH_OUT_JSONL = OUT_DIR / "preds.jsonl"

# --- Labels MVP ---
MVP_LABELS = ["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]

# --- GPU ---
if torch.cuda.is_available():
    print("✅ GPU:", torch.cuda.get_device_name(0))
    DEVICE = 0
    DTYPE = torch.float16
else:
    print("⚠️ No GPU, usarás CPU (lento).")
    DEVICE = -1
    DTYPE = torch.float32


In [ ]:
#Celda 2: elegimos el  modelo 
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0} if DEVICE == 0 else None,
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=DEVICE,
)
print("OK model:", MODEL_ID)


In [ ]:
#cargamos modelo en 4-bit sin offload, forzando GPU. Esto evitará el error de dispatch CPU/disk 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},   # <- fuerza TODO a GPU:0, evita offload automático
)

model.eval()
print("Modelo cargado OK en GPU")


In [ ]:
#comprobamos que el modelo quedó totalmente en GPU
import torch
print("cuda:", torch.cuda.is_available())
print("gpu:", torch.cuda.get_device_name(0))
print("mem free/total:", torch.cuda.mem_get_info())
print("model device:", model.device)



In [ ]:
#verificamos que no hay offload y todo está en GPU 
# Si todo está en GPU, no debería haber parámetros en cpu
on_cpu = 0
total = 0
for p in model.parameters():
    total += 1
    if p.device.type == "cpu":
        on_cpu += 1
print("params on CPU:", on_cpu, "/", total)


In [ ]:
#fijamos PAD/EOS y evitamos warnings de attention_mask y padding (no utilizamos) 
# Asegurar tokens especiales para generación estable
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("eos_token_id:", tokenizer.eos_token_id)
print("pad_token_id:", tokenizer.pad_token_id)


In [ ]:
#generación con attention_mask + pad_token_id. Ejecutamos run_llm 
import torch

@torch.no_grad()
def run_llm(system: str, user: str, max_new_tokens: int = 256) -> str:
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True
    )
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc["attention_mask"].to(model.device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        top_p=1.0,
        pad_token_id=tokenizer.pad_token_id,  # <- ya definido
        eos_token_id=tokenizer.eos_token_id,
    )

    # Solo devolvemos lo generado (sin el prompt)
    gen_ids = output_ids[0, input_ids.shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [ ]:
print(run_llm("Eres un asistente.", 'Devuélveme SOLO este JSON: {"doc_uid":"test","spans":[]}'))


**¿Por qué hacemos eso?**
- Los modelos chat (Llama 3.1 Instruct) usan plantillas de conversación.
- Para que generate() sea determinista y sin padding raro, conviene: definir pad_token
- pasar attention_mask y decodificar solo la parte generada (no el prompt completo)

Eso es exactamente lo que hace la versión corregida.

Siguiente paso

Al confirmar que el test ya saca solo JSON limpio ({"doc_uid":"test","spans":[]}), pasamos a extract_first_json + Pydantic.
Con esta nueva run_llm, el parser fallará mucho menos porque el modelo devolverá menos “basura” alrededor del JSON.

In [ ]:
#parseo JSON robusto 
import json
import re
import json

def extract_root_json(text: str) -> dict:
    """
    Extrae el objeto JSON raíz (completo) del output.
    Implementa balanceo de llaves para soportar anidamiento.
    Devuelve el primer objeto parseable que contenga doc_uid y spans.
    """
    if not isinstance(text, str):
        raise ValueError("Output no es string")

    candidates = []
    start_positions = [i for i, ch in enumerate(text) if ch == "{"]

    for start in start_positions:
        depth = 0
        for end in range(start, len(text)):
            ch = text[end]
            if ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    chunk = text[start:end+1]
                    # intentamos parsear
                    try:
                        obj = json.loads(chunk)
                        # preferimos el root esperado
                        if isinstance(obj, dict) and ("doc_uid" in obj) and ("spans" in obj):
                            return obj
                        candidates.append(obj)
                    except json.JSONDecodeError:
                        pass
                    break

    # fallback: si alguno parseó pero no tenía keys, devolvemos el primero dict parseable
    for obj in candidates:
        if isinstance(obj, dict):
            return obj

    raise ValueError("No se pudo extraer JSON raíz válido con doc_uid y spans.")



In [ ]:
#normalizacion de etiquetas 
from typing import List, Literal, Optional
from pydantic import BaseModel, Field, ValidationError, field_validator

MVP_LABELS = {
    "OBJETO",
    "PRECIO_DEL_CONTRATO",
    "DURACION_TOTAL_DEL_CONTRATO",
    "RESOLUCION",
}

class SpanPred(BaseModel):
    tag: Literal["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]
    start: int = Field(ge=0)
    end: int = Field(ge=0)

    @field_validator("end")
    @classmethod
    def end_after_start(cls, v, info):
        start = info.data.get("start")
        if start is not None and v < start:
            raise ValueError("end must be >= start")
        return v

class PredDoc(BaseModel):
    doc_uid: str
    spans: List[SpanPred] = Field(default_factory=list)

def validate_pred(doc_uid: str, text: str, pred_json: dict) -> PredDoc:
    """
    - valida esquema
    - valida límites start/end dentro del texto
    - valida que substring no sea vacío/solo espacios
    - filtra etiquetas fuera del MVP
    """
    pred = PredDoc.model_validate(pred_json)

    fixed_spans = []
    for s in pred.spans:
        if s.tag not in MVP_LABELS:
            continue
        if s.start < 0 or s.end > len(text):
            continue
        substr = text[s.start:s.end]
        if not substr.strip():
            continue
        fixed_spans.append(s)

    pred.spans = fixed_spans
    # Forzamos doc_uid del contexto (por si el modelo la cambia)
    pred.doc_uid = doc_uid
    return pred


In [ ]:
import json
from pathlib import Path

SPLITS_DIR = ROOT / "outputs" / "splits"

def load_jsonl(path):
    rows = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    return rows

gold_val = load_jsonl(SPLITS_DIR / "gold_val.jsonl")
gold_test = load_jsonl(SPLITS_DIR / "gold_test.jsonl")  # opcional ahora

print("gold_val:", len(gold_val))
print("gold_test:", len(gold_test))


In [ ]:
  #para evitar errores de SYSTEM not defined
SYSTEM = """Eres un modelo de ANOTACIÓN de texto. Tu tarea es marcar spans (fragmentos) dentro de un texto.

Salida obligatoria: JSON estricto con:
- doc_uid: string
- spans: lista de objetos {tag, start, end}

Reglas de offsets:
- start/end son offsets de caracteres 0-based sobre el texto EXACTO.
- end es exclusivo.
- El substring text[start:end] debe existir exactamente.
- No inventes texto ni offsets.
- Puedes devolver múltiples spans.
- Devuelve como máximo 8 spans en total.
- Si hay más candidatos, elige los más claros y completos.

Etiquetas permitidas (solo estas 4):
OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION

Devuelve SOLO JSON, sin explicaciones ni texto adicional.
"""


In [ ]:
#wrapper de prediccion seguro 
def safe_predict_one(doc_uid: str, text: str, user_prompt: str):
    assert "SYSTEM" in globals(), "SYSTEM no está definido (ejecuta la celda de SYSTEM)"
    raw = run_llm(SYSTEM, user_prompt, max_new_tokens=900)
    pred_json = extract_first_json(raw)
    pred_obj = validate_pred(doc_uid, text, pred_json)
    return pred_obj.model_dump(), raw

In [ ]:
d0 = gold_val[0]
raw = run_llm(
    SYSTEM,
    f'Devuelve SOLO este JSON: {{"doc_uid":"{d0["doc_uid"]}","spans":[]}}',
    max_new_tokens=200
)

print("RAW LEN:", len(raw))
print("RAW REPR:", repr(raw[:500]))


In [ ]:
def safe_predict_one(doc_uid: str, text: str, user_prompt: str):
    raw = run_llm(SYSTEM, user_prompt, max_new_tokens=900)

    # Manejo de rechazo/no-JSON
    if "{" not in raw:
        return {"doc_uid": doc_uid, "spans": [], "_error": "non_json_output", "_raw": raw}, raw
  
    pred_json = extract_root_json(raw)
    pred_obj = validate_pred(doc_uid, text, pred_json)
    return pred_obj.model_dump(), raw


In [ ]:
SYSTEM_SANITY = "Eres un asistente que devuelve exactamente el JSON solicitado. Devuelve SOLO JSON, sin texto adicional."

d0 = gold_val[0]
raw = run_llm(
    SYSTEM_SANITY,
    f'Devuelve EXACTAMENTE este JSON: {{"doc_uid":"{d0["doc_uid"]}","spans":[]}}',
    max_new_tokens=60
)

print("RAW:", raw)
print(extract_first_json(raw))


In [ ]:
SYSTEM = """Eres un modelo de ANOTACIÓN de texto. Tu tarea es marcar spans (fragmentos) dentro de un texto.

Salida obligatoria: JSON estricto con:
- doc_uid: string
- spans: lista de objetos {tag, start, end}

Reglas de offsets:
- start/end son offsets de caracteres 0-based sobre el texto EXACTO.
- end es exclusivo.
- El substring text[start:end] debe existir exactamente.
- No inventes texto ni offsets.
- Puedes devolver múltiples spans.
- Devuelve como máximo 8 spans en total.
- Si hay más candidatos, elige los más claros y completos.


Etiquetas permitidas (solo estas 4):
OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION

Devuelve SOLO JSON, sin explicaciones ni texto adicional.
"""



In [ ]:
print(SYSTEM[:800])


In [ ]:
#sanity check rápido con un doc real (si tenemos gold_val)
d0 = gold_val[0]
pred0, raw0 = safe_predict_one(
    d0["doc_uid"],
    d0["text"],
    f'Devuelve SOLO este JSON: {{"doc_uid":"{d0["doc_uid"]}","spans":[]}}'
)
pred0


In [ ]:
d0 = gold_val[0]
pred0, raw0 = safe_predict_one(
    d0["doc_uid"],
    d0["text"],
    f'Devuelve EXACTAMENTE este JSON: {{"doc_uid":"{d0["doc_uid"]}","spans":[]}}'
)
pred0


In [ ]:
#Dejamos safe_predict_one robusto ante no-JSON (imprescindible para batch) 
def safe_predict_one(doc_uid: str, text: str, user_prompt: str):
    raw = run_llm(SYSTEM, user_prompt, max_new_tokens=900)

    # Si no devuelve JSON, no rompemos el experimento
    if "{" not in raw:
        return {"doc_uid": doc_uid, "spans": [], "_error": "non_json_output", "_raw": raw}, raw

    pred_json = extract_first_json(raw)
    pred_obj = validate_pred(doc_uid, text, pred_json)
    return pred_obj.model_dump(), raw


## Construimos memory_str y lanzamos Exp1/Exp2/Exp3

In [ ]:
#al resetear kernel quizá haya celdas que no han persistido. Nos asgeuramos re-ejecutando: 
import json
from pathlib import Path

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows


In [ ]:
memory_selected = load_json(PATH_MEMORY)
print("memory_selected:", len(memory_selected))
print("keys:", memory_selected[0].keys())
print("example:", memory_selected[0])


**_memory_selected_ tiene offsets, pero no trae el texto del span. Así que el siguiente paso es:**
- Cargar gold (corpus anotado completo)
- Construir un índice doc_uid -> text
- “Adjuntar” text = gold_text[start:end] a cada ejemplo de memoria
- Renderizar memory_str para Exp2/Exp3
- (Opcional) crear fewshot_extra automático para Exp3


In [ ]:
#cargamos gold completo+indexamos por doc_uid 
# --- Cargar gold completo (para poder extraer el texto de cada span de memoria) ---
if not PATH_GOLD.exists():
    raise FileNotFoundError(f"No encuentro {PATH_GOLD}")

gold_raw = load_jsonl(PATH_GOLD) if is_jsonl(PATH_GOLD) else load_json(PATH_GOLD)
print("gold_raw docs:", len(gold_raw))

# Index doc_uid -> text
gold_text_by_uid = {}
missing_uid = 0

for d in gold_raw:
    txt = d.get("text", "")
    if not txt:
        continue
    uid = stable_uid(txt)
    # si hay colisiones (muy raro), mantenemos el primero
    gold_text_by_uid.setdefault(uid, txt)

print("gold_text_by_uid:", len(gold_text_by_uid))

# Sanity: ¿todos los doc_uid de memoria existen en gold?
mem_uids = {ex["doc_uid"] for ex in memory_selected}
missing = [u for u in mem_uids if u not in gold_text_by_uid]
print("memory doc_uids:", len(mem_uids))
print("missing in gold_text_by_uid:", len(missing))
if missing:
    print("Ejemplo missing:", missing[0])


In [ ]:
#adjuntamos texto a memoria+validamos offsets 
def attach_span_text(memory_selected, gold_text_by_uid):
    out = []
    bad = 0

    for ex in memory_selected:
        uid = ex["doc_uid"]
        text = gold_text_by_uid.get(uid)
        if text is None:
            bad += 1
            continue

        start, end = ex["start"], ex["end"]
        if not (0 <= start <= end <= len(text)):
            bad += 1
            continue

        span_txt = text[start:end]
        if not span_txt.strip():
            bad += 1
            continue

        out.append({
            **ex,
            "text": span_txt
        })

    return out, bad

memory_selected_with_text, bad_mem = attach_span_text(memory_selected, gold_text_by_uid)

print("memory_selected_with_text:", len(memory_selected_with_text))
print("bad memory examples:", bad_mem)
print("Ejemplo con text:", memory_selected_with_text[0])


In [ ]:
#renderizamos memory_str (para prompts Exp2/Exp3 
def render_memory_blocks(examples):
    # formato compacto y estable para prompting
    lines = []
    for ex in examples:
        lines.append(
            f'- label: {ex["label"]}\n'
            f'  criterion: {ex["criterion"]}\n'
            f'  example: "{ex["text"]}"'
        )
    return "\n".join(lines)

memory_str = render_memory_blocks(memory_selected_with_text)
print("memory_str chars:", len(memory_str))
print(memory_str[:600])


**(Opcional pero recomendado) — fewshot_extra automático para Exp3 (4 ejemplos, 1 por etiqueta)**
- Esto crea 4 ejemplos extra distintos de los 16 de memoria, sacados del gold, para dar “variedad formal” en Exp3

In [ ]:
# Construimos un pool de spans MVP desde gold_raw (como hiciste antes en NB03)
def filter_tags_mvp(tags):
    return [t for t in tags if t.get("tag") in MVP_LABELS]

gold_spans_pool = []
for d in gold_raw:
    txt = d.get("text", "")
    if not txt:
        continue
    uid = stable_uid(txt)
    tags = filter_tags_mvp(d.get("tags", []))
    for t in tags:
        start, end = t["start"], t["end"]
        if 0 <= start <= end <= len(txt):
            stxt = txt[start:end]
            if stxt.strip():
                gold_spans_pool.append({
                    "label": t["tag"],
                    "criterion": "FEWSHOT_EXTRA (variedad formal)",
                    "doc_uid": uid,
                    "start": start,
                    "end": end,
                    "text": stxt
                })

print("Pool spans MVP:", len(gold_spans_pool))

# Excluimos doc_uids ya usados en memoria para evitar leakage dentro del prompt
mem_doc_uids = {ex["doc_uid"] for ex in memory_selected_with_text}
pool_no_memdocs = [x for x in gold_spans_pool if x["doc_uid"] not in mem_doc_uids]

# Selección 1 por etiqueta
fewshot_extra = []
for lab in MVP_LABELS:
    cand = [x for x in pool_no_memdocs if x["label"] == lab]
    # elegimos el primero (o aleatorio reproducible)
    if cand:
        fewshot_extra.append(cand[0])

print("fewshot_extra:", len(fewshot_extra))
for ex in fewshot_extra:
    print(ex["label"], ex["doc_uid"][:12], ex["text"][:100])

fewshot_str = render_memory_blocks(fewshot_extra)


In [ ]:
#ajustamos los prompts Exp1/Exp2/Exp3 (ya con memory_str y fewshot_str) 
LABEL_DEFS_MIN = """
Etiquetas:
- OBJETO: prestación principal / objeto del contrato, título o descripción concisa.
- PRECIO_DEL_CONTRATO: cuantía/precio del contrato (importe total, anualidad, con o sin IVA si aplica).
- DURACION_TOTAL_DEL_CONTRATO: periodo total de vigencia/ejecución, plazos, fechas de inicio/fin.
- RESOLUCION: causas/condiciones de resolución/extinción del contrato.
"""

def make_user_prompt_exp1(doc_uid, text):
    text_win = make_text_window(text)   #ventanas de texto (Acortar)
    return f"... TEXTO: {text_win} ..."


TEXTO (doc_uid={doc_uid}):
\"\"\"{text}\"\"\"

Devuelve SOLO JSON con spans.
"""

def make_user_prompt_exp2(doc_uid: str, text: str) -> str:
    return f"""{LABEL_DEFS_MIN}

Guía (criterios + ejemplos):
{memory_str}

TEXTO (doc_uid={doc_uid}):
\"\"\"{text}\"\"\"

Devuelve SOLO JSON con spans.
"""

def make_user_prompt_exp3(doc_uid: str, text: str) -> str:
   return f"""{LABEL_DEFS_MIN}

Guía (criterios + ejemplos):
{memory_str}

Few-shot extra:
{fewshot_str}

TEXTO (doc_uid={doc_uid}):
\"\"\"{text}\"\"\"

Devuelve SOLO JSON con spans.
"""


In [ ]:
#diagnostico inmediato_ ver el raw que rompió 
def predict_for_docs(docs, exp_name: str, make_prompt_fn):
    out_rows = []
    bad = 0
    for i, d in enumerate(docs):
        doc_uid = d["doc_uid"]
        text = d["text"]
        user_prompt = make_prompt_fn(doc_uid, text)

        try:
            pred, raw = safe_predict_one(doc_uid, text, user_prompt)
        except Exception as e:
            print("\n" + "="*120)
            print("ERROR en", exp_name, "doc", i, "doc_uid", doc_uid)
            print("Exception:", repr(e))
            print("RAW (primeros 1200 chars):")
            raw_dbg = run_llm(SYSTEM, user_prompt, max_new_tokens=900)
            print(raw_dbg[:1200])
            print("="*120 + "\n")
            raise

        if isinstance(pred, dict) and pred.get("_error"):
            bad += 1
        out_rows.append(pred)

        if (i+1) % 5 == 0:
            print(f"{exp_name}: {i+1}/{len(docs)} | errores: {bad}")
    print(exp_name, "DONE. total:", len(docs), "errores:", bad)
    return out_rows


In [ ]:
#hacemos safe_predict_one aún mas tolerante (no romper batch) 
def safe_predict_one(doc_uid: str, text: str, user_prompt: str):
    raw = run_llm(SYSTEM, user_prompt, max_new_tokens=150)

    if "{" not in raw:
        return {"doc_uid": doc_uid, "spans": [], "_error": "non_json_output", "_raw": raw}, raw

    try:
        pred_json = extract_first_json(raw)
    except Exception as e:
        return {"doc_uid": doc_uid, "spans": [], "_error": "json_parse_error", "_raw": raw, "_exception": repr(e)}, raw

    try:
        pred_obj = validate_pred(doc_uid, text, pred_json)
        return pred_obj.model_dump(), raw
    except Exception as e:
        return {"doc_uid": doc_uid, "spans": [], "_error": "validation_error", "_raw": raw, "_pred_json": pred_json, "_exception": repr(e)}, raw


In [ ]:
def try_autoclose_json(raw: str) -> str:
    # Si empieza por "{" pero no termina en "}", intentamos cerrar de forma conservadora
    s = raw.strip()
    if not s.startswith("{"):
        return raw
    # cierre simple si falta
    if s.count("{") > s.count("}"):
        s = s + ("\n}" * (s.count("{") - s.count("}")))
    if s.count("[") > s.count("]"):
        s = s + ("]" * (s.count("[") - s.count("]")))
    return s


In [ ]:
raw2 = try_autoclose_json(raw)
pred_json = extract_first_json(raw2)


In [ ]:
# Diagnóstico mínimo: imprime 2 raws de error (solo una vez)
# Debug: ver qué devuelve cuando NO hay JSON
for i in range(3):
    d = gold_val[i]
    doc_uid = d["doc_uid"]
    text = d["text"]
    user_prompt = make_user_prompt_exp1(doc_uid, text)
    raw = run_llm(SYSTEM, user_prompt, max_new_tokens=400)
    print("\n" + "="*120)
    print("DOC", i, "doc_uid", doc_uid)
    print("RAW REPR:", repr(raw[:400]))


**Este output indica que sí hay formato**

**Arreglo fuerte: cambia Exp1 para forzar JSON (como el sanity)**

El Exp1 actual probablemente es demasiado “explicativo”. Hazlo ultra-rígido y añade un ejemplo mínimo dentro del prompt (no es few-shot de contenido, solo de formato).

Reemplaza completo make_user_prompt_exp1 por esto:

In [ ]:
def make_user_prompt_exp1(doc_uid: str, text: str) -> str:
    return f"""
TAREA:
Anota spans del texto para estas etiquetas:
OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION.

{FORMAT_RULES}

TEXTO (doc_uid={doc_uid}):
\"\"\"{text}\"\"\"

Devuelve SOLO este JSON (rellenando spans si procede):
{{"doc_uid":"{doc_uid}","spans":[]}}
"""



**Ajuste en run_llm: fuerza “eos” y evita respuestas largas**

A veces el modelo genera “texto” si no ve el final. Podemos ayudar:
- mantén do_sample=False
- subimos max_new_tokens (1500), antes teníamos 600-900 y dio error
- (opcional) añade repetition_penalty=1.05 para evitar divagar

In [ ]:
#para distinguir de dónde viene el error, probamos con 3 tipos: 


## Ejecutamos Exp1/Exp2/Exp3 

In [ ]:
FORMAT_RULES = """
REGLAS OBLIGATORIAS:
- Devuelve SOLO JSON válido.
- Usa comillas dobles en claves y strings.
- No devuelvas texto fuera del JSON.
- Devuelve como máximo 12 spans en total.
- Solo anota spans que correspondan claramente a las etiquetas.
- No “partas” el texto en trozos consecutivos (NO segmentación artificial).
- Si no encuentras evidencia clara para una etiqueta, no la inventes.
- Los offsets deben corresponder exactamente al substring del texto.
- Devuelve como máximo 8 spans en total.
- Si hay más candidatos, elige los más claros y completos.
"""


In [ ]:
#ejecutamos Exp1/Exp2/Exp3 sobre gold_val y guardamos 
def save_jsonl(rows, path: Path):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def predict_for_docs(docs, exp_name: str, make_prompt_fn):
    out_rows = []
    bad = 0
    for i, d in enumerate(docs):
        doc_uid = d["doc_uid"]
        text = d["text"]
        user_prompt = make_prompt_fn(doc_uid, text)

        pred, raw = safe_predict_one(doc_uid, text, user_prompt)
        if isinstance(pred, dict) and pred.get("_error"):
            bad += 1
        out_rows.append(pred)

        if (i+1) % 5 == 0:
            print(f"{exp_name}: {i+1}/{len(docs)} | errores_no_json: {bad}")
    print(exp_name, "DONE. total:", len(docs), "errores_no_json:", bad)
    return out_rows

pred_exp1_val = predict_for_docs(gold_val, "exp1_val", make_user_prompt_exp1)
pred_exp2_val = predict_for_docs(gold_val, "exp2_val", make_user_prompt_exp2)
pred_exp3_val = predict_for_docs(gold_val, "exp3_val", make_user_prompt_exp3)

save_jsonl(pred_exp1_val, OUT_PRED / "exp1_val.jsonl")
save_jsonl(pred_exp2_val, OUT_PRED / "exp2_val.jsonl")
save_jsonl(pred_exp3_val, OUT_PRED / "exp3_val.jsonl")

print("Guardado en:", OUT_PRED)


Esto indica algo muy concreto: casi todos los outputs están entrando por la rama “no JSON / no {” (tu contador lo llama errores_no_json). O sea: el modelo está devolviendo texto sin llaves {...} para la mayoría de documentos.

Esto suele pasar cuando el prompt “real” (Exp1) está activando rechazo / respuesta conversacional, mientras que el sanity check “Devuelve EXACTAMENTE…” sí funciona.

Vamos a arreglarlo con dos medidas:
- Cambiar el prompt de Exp1 para que sea tan “cerrado” como el sanity check, y
- Registrar el raw de los primeros errores para ver el patrón.

Lo que te está pasando ahora es casi seguro una de estas dos cosas:

1. Tu contador “no_json” está mal (está contando cualquier _error, no solo non_json_output), o
2. En batch, para muchos docs el modelo vuelve a responder tipo “No puedo ayudarte…” (rechazo) o devuelve algo vacío, y eso sí dispara non_json_output.

Vamos a salir de dudas en 2 minutos con una celda de instrumentación que:
- cuenta errores por tipo,
- imprime el primer raw que cae en non_json_output,
- y guarda ese raw a disco para inspección.

In [ ]:
#Instrumentación: contador real + primer ejemplo de fallo (solo debug) 
from collections import Counter

def predict_for_docs_debug(docs, exp_name: str, make_prompt_fn, max_new_tokens=1200):
    out_rows = []
    err_counter = Counter()
    first_nonjson = None
    first_parse = None
    first_val = None

    for i, d in enumerate(docs):
        doc_uid = d["doc_uid"]
        text = d["text"]
        user_prompt = make_prompt_fn(doc_uid, text)

        pred, raw = safe_predict_one(doc_uid, text, user_prompt)

        if isinstance(pred, dict) and pred.get("_error"):
            err_counter[pred["_error"]] += 1

            if pred["_error"] == "non_json_output" and first_nonjson is None:
                first_nonjson = (i, doc_uid, raw)
            if pred["_error"] == "json_parse_error" and first_parse is None:
                first_parse = (i, doc_uid, raw)
            if pred["_error"] == "validation_error" and first_val is None:
                first_val = (i, doc_uid, raw)

        out_rows.append(pred)

        if (i+1) % 5 == 0:
            print(f"{exp_name}: {i+1}/{len(docs)} | errors:", dict(err_counter))

    print(exp_name, "DONE. errors:", dict(err_counter))

    # Imprime 1 ejemplo de cada tipo (si existe)
    def show(label, item):
        if item is None:
            print(f"\nNo hubo {label}.")
            return
        i, uid, raw = item
        print("\n" + "="*120)
        print(label, "| doc idx:", i, "| doc_uid:", uid)
        print("RAW LEN:", len(raw))
        print("RAW (primeros 800 chars):")
        print(raw[:800])

    show("FIRST non_json_output", first_nonjson)
    show("FIRST json_parse_error", first_parse)
    show("FIRST validation_error", first_val)

    return out_rows

# Ejecuta SOLO exp1 con debug primero
pred_exp1_val = predict_for_docs_debug(gold_val, "exp1_val", make_user_prompt_exp1)


## 'validation_error'
El problema viene de offsets inventados (lo arreglaremos obligando a copiar el substring y/o bajando ambición en el system)

validation_error porque tu extract_first_json() está extrayendo el JSON equivocado: en vez de devolver el objeto raíz {"doc_uid":..., "spans":[...]}, está devolviendo un objeto interno tipo {"tag":"PRECIO_DEL_CONTRATO","start":114,"end":141}. Por eso Pydantic dice que falta doc_uid.

 34 líneas: son 34 porque gold_val.jsonl tiene 34 documentos. Eso corresponde al split de validación: en NB02 hiciste val_n = int(0.10 * len(rest)) y tras filtrar leakage te quedaron 34 docs en validación (y otros 34 en test).
 
Siguientes pasos:
1. Arreglo crítico: reemplazar extract_first_json para que SIEMPRE devuelva el objeto raíz
- Tu regex anterior re.findall(r"\{[\s\S]*?\}", ...) no puede manejar JSON anidado; se corta en la primera } que ve y te devuelve objetos internos.
- Usa este extractor por balanceo de llaves (stack) y además prioriza el objeto que contiene "doc_uid" y "spans".

2. Arreglo secundario: tus _raw están truncados (muchos no cierran)
- En tu log, varios _raw acaban en ... "end": o se corta a mitad. Eso dará json_parse_error (no validation_error) cuando el extractor sea correcto.
- Para reducir truncado:
    - Limita el número de spans.
    - Sube max_new_tokens.
    - Recorta texto de entrada (porque estás pasando contratos largos).

3. Limitar spans en el prompt

Añade esto a todos tus prompts:
- Devuelve como máximo 8 spans en total.
- Si hay más candidatos, elige los más claros y completos.

4. Recortar el texto (clave)
- Para MVP de prompting, NO metas el contrato completo si mide miles de chars. Haz ventana con heurística:
- Para EXP1: coge primeros 2500 chars + últimos 2500 chars (muchas resoluciones van al final).
- O bien busca keywords para extraer fragmentos (OBJETO/Precio/Duración/Resolución).

In [ ]:
#recortamos texto para probar Exp1
def make_text_window(text: str, head: int = 2500, tail: int = 2500) -> str:
    if len(text) <= head + tail + 200:
        return text
    return text[:head] + "\n...\n" + text[-tail:]


In [ ]:
#confirmamos el arreglo con 1 doc 
d0 = gold_val[0]
raw = run_llm(SYSTEM, make_user_prompt_exp1(d0["doc_uid"], d0["text"]), max_new_tokens=1500)
pred_json = extract_root_json(raw)
pred_json.keys(), len(pred_json.get("spans", []))


In [ ]:
validate_pred(d0["doc_uid"], d0["text"], pred_json)


In [ ]:
# ===============================
# NB4 — Carga segura del modelo
# ===============================

import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

# --------- Sanity checks ---------
print("Python:", torch.__file__)
print("CUDA disponible:", torch.cuda.is_available())
if not torch.cuda.is_available():
    raise RuntimeError("❌ CUDA no disponible. Este NB requiere GPU.")

print("GPU:", torch.cuda.get_device_name(0))

# --------- Modelo ---------
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# --------- Config 4-bit (estable) ---------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# --------- Tokenizer ---------
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
)

# Llama necesita padding explícito
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------- Modelo ---------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    dtype=torch.float16,
    device_map={"": 0},  # fuerza TODO a GPU
)

model.eval()

print("✅ Modelo cargado correctamente en GPU (4-bit)")


In [ ]:
import torch

print("Modelo cargado:", model.__class__.__name__)
print("Dispositivo primer parámetro:", next(model.parameters()).device)
print("dtype primer parámetro:", next(model.parameters()).dtype)
print("Memoria GPU (GB):",
      round(torch.cuda.memory_allocated()/1024**3, 2), "alloc |",
      round(torch.cuda.memory_reserved()/1024**3, 2), "reserved")


In [ ]:
#Checklist mínimo antes de inferencia
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
print("bf16:", torch.cuda.is_bf16_supported())



In [ ]:
import torch

def generate_chat(model, tokenizer, system: str, user: str,
                  max_new_tokens: int = 512,
                  temperature: float = 0.0,
                  top_p: float = 0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,  # evita warnings y edge cases
    )

    # sampling solo si temperature > 0
    if temperature and temperature > 0:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **gen_kwargs
        )

    gen_ids = out[0, input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()



In [ ]:
#Monta el prompt del proyecto (system + user)
#System (rol + conducta)
SYSTEM = """Eres un anotador jurídico experto en contratos del sector público en España.
Tu tarea es detectar spans exactos en el texto para estas etiquetas:
- OBJETO
- PRECIO_DEL_CONTRATO
- DURACION_TOTAL_DEL_CONTRATO
- RESOLUCION

Devuelve SIEMPRE JSON válido y nada más. No inventes texto. No cambies el texto original.
Los offsets start/end son índices de caracteres sobre el texto original (Python slicing, end exclusivo).
Si no encuentras una evidencia clara para una etiqueta, no la incluyas en la lista spans. 
"""


In [ ]:
#Inyectar la memoria (few-shot) desde tu JSON curado
#Tu memoria buena ahora es outputs/memory/memory_selected_CURATED.json
import json
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
MEM_PATH = ROOT / "outputs" / "memory" / "memory_selected_CURATED.json"
GOLD_PATH = ROOT / "gold" / "corpus_annotated.jsonl"  # ajusta si usas json

memory_selected = json.loads(MEM_PATH.read_text(encoding="utf-8"))
print("Memoria ejemplos:", len(memory_selected))

In [ ]:
#Ahora necesitamos convertir doc_uid+offsets a texto (para meter el ejemplo literal en el prompt). Para eso abrimos el gold y construimos un uid_to_text.
import hashlib, json

def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

uid_to_text = {}
# si es jsonl:
with open(GOLD_PATH, "r", encoding="utf-8") as f:
    for line in f:
        d = json.loads(line)
        txt = d.get("text","")
        if txt:
            uid_to_text[stable_uid(txt)] = txt

def render_memory_block(memory_selected, uid_to_text, k=None):
    items = memory_selected if k is None else memory_selected[:k]
    blocks = []
    for ex in items:
        txt = uid_to_text[ex["doc_uid"]]
        span = txt[ex["start"]:ex["end"]].replace("\n", " ")
        blocks.append(
            f"- label: {ex['label']}\n"
            f"  criterio: {ex['criterion']}\n"
            f"  ejemplo: {span}"
        )
    return "\n".join(blocks)

MEMORY_BLOCK = render_memory_block(memory_selected, uid_to_text)
print(MEMORY_BLOCK[:2000])


In [ ]:
#User prompt final (instrucciones + memoria + texto a etiquetar + formato)
import json

JSON_SCHEMA_HINT = """Devuelve un JSON con esta forma:
{
  "spans": [
    {"label": "...", "start": 0, "end": 10}
  ]
}
Reglas:
- start/end deben delimitar substring NO vacío
- end es exclusivo
- no duplicar spans idénticos
- si no hay spans, devuelve {"spans": []}
"""

def build_user_prompt(text: str) -> str:
    return f"""MEMORIA (ejemplos correctos):
{MEMORY_BLOCK}

TAREA:
Anota en el TEXTO los spans para las 4 etiquetas: OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION.

{JSON_SCHEMA_HINT}

TEXTO:
\"\"\"{text}\"\"\"
"""


In [ ]:
#Prueba rápida con 1 documento
# coge un doc cualquiera del gold o de tu split de evaluación
sample_text = next(iter(uid_to_text.values()))
user_prompt = build_user_prompt(sample_text)

resp = generate_chat(model, tokenizer, SYSTEM, user_prompt, max_new_tokens=600, temperature=0.0)
print(resp[:2000])


In [ ]:
#Post-proceso mínimo para limpiar spans inválidos
import json

def clean_spans(pred: dict, text: str):
    spans = pred.get("spans", [])
    cleaned = []
    for s in spans:
        lab = s.get("label")
        start = int(s.get("start", -1))
        end = int(s.get("end", -1))

        # descarta vacíos / fuera de rango
        if start < 0 or end < 0 or start >= end:
            continue
        if end > len(text):
            continue

        cleaned.append({"label": lab, "start": start, "end": end})
    return {"spans": cleaned}


In [ ]:
raw = generate_chat(model, tokenizer, SYSTEM, user_prompt, max_new_tokens=600, temperature=0.0)
pred = json.loads(raw)
pred = clean_spans(pred, sample_text)
pred


In [ ]:
def generate_plain(model, tokenizer, prompt: str, max_new_tokens=512, temperature=0.0, top_p=0.9):
    enc = tokenizer(prompt, return_tensors="pt")
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    gen_kwargs = dict(max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
    if temperature and temperature > 0:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    with torch.no_grad():
        out = model.generate(input_ids=input_ids, attention_mask=attention_mask, **gen_kwargs)

    gen_ids = out[0, input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

def build_plain_prompt(system: str, user: str) -> str:
    return f"SYSTEM:\n{system}\n\nUSER:\n{user}\n\nASSISTANT:\n"


In [ ]:
prompt = build_plain_prompt(SYSTEM, user_prompt)
resp = generate_plain(model, tokenizer, prompt, max_new_tokens=600, temperature=0.0)
print(resp[:2000])


In [ ]:
#Celda 1 — Imports + rutas + carga prompt_regression
import os, json, re
from pathlib import Path
from typing import List, Dict, Any, Optional

ROOT = Path("/home/jovyan/inesagent")
PATH_PROMPT_REG = ROOT / "outputs" / "splits" / "prompt_regression.jsonl"

OUT_DIR = ROOT / "outputs" / "predictions"
OUT_DIR.mkdir(parents=True, exist_ok=True)

assert PATH_PROMPT_REG.exists(), f"No existe {PATH_PROMPT_REG}"

def load_jsonl(path: Path) -> List[Dict[str, Any]]:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

prompt_reg_docs = load_jsonl(PATH_PROMPT_REG)
print("prompt_regression docs:", len(prompt_reg_docs))
print("keys ejemplo:", list(prompt_reg_docs[0].keys()))
print("len(text) ejemplo:", len(prompt_reg_docs[0]["text"]))


In [ ]:
#Celda 2 — Carga memoria final + bloqueo anti-leakage
PATH_MEMORY = ROOT / "outputs" / "memory" / "memory_selected_FINAL.json"
PATH_BLOCKED = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

memory = json.loads(PATH_MEMORY.read_text(encoding="utf-8"))
blocked_uids = set(json.loads(PATH_BLOCKED.read_text(encoding="utf-8")))

print("memory examples:", len(memory))
print("blocked uids:", len(blocked_uids))
print("memory[0] keys:", list(memory[0].keys()))


In [ ]:
#Celda 2.1 (antes de pydantic) Haz el parseo “tolerante” antes de Pydantic (convierte spans vacíos → se eliminan)
#Antes hacíamos Pred.model_validate_json(raw) directamente → si el modelo devuelve basura, muere.
#Ahora: mostramos el RAW siempre (para depurar),parseamos JSON,limpiamos spans inválidos (0,0, quote vacío),y Pydantic valida lo que queda
def _extract_json_object(s: str) -> str:
    # coge el último {...} del final
    m = re.search(r"\{.*\}\s*$", s, flags=re.DOTALL)
    return m.group(0) if m else s

def _sanitize_pred_dict(obj: Dict[str, Any]) -> Dict[str, Any]:
    # Normaliza casos típicos de “plantilla vacía”
    spans = obj.get("spans", [])
    if not isinstance(spans, list):
        return {"spans": []}

    cleaned = []
    for sp in spans:
        if not isinstance(sp, dict):
            continue
        start = sp.get("start", None)
        end = sp.get("end", None)
        quote = sp.get("quote", None)
        label = sp.get("label", None)

        # Descarta spans vacíos / mal formados
        if label not in LABELS:
            continue
        if not isinstance(start, int) or not isinstance(end, int):
            continue
        if end <= start:
            continue
        if not isinstance(quote, str) or not quote.strip():
            continue

        cleaned.append({"label": label, "start": start, "end": end, "quote": quote})

    return {"spans": cleaned}

def run_one_exp(text: str, exp: int):
    if exp == 1:
        user = build_user_exp1(text)
    elif exp == 2:
        user = build_user_exp2(text, memory_block)
    elif exp == 3:
        user = build_user_exp3(text, memory_block, fewshot_extra)
    else:
        raise ValueError("exp debe ser 1,2,3")

    raw = generate_chat(model, tokenizer, SYSTEM_BASE, user, max_new_tokens=600, temperature=0.0)
    raw_json = _extract_json_object(raw)

    print("RAW (primeros 1200 chars):\n", raw[:1200], "\n")

    # 1) parse JSON “a mano”
    obj = json.loads(raw_json)
    obj = _sanitize_pred_dict(obj)

    # 2) valida con Pydantic
    pred = Pred.model_validate(obj)
    pred2 = strict_verify(pred, text)

    print("Spans validados:", len(pred2.spans))
    for s in pred2.spans:
        print("\n", s.label, s.start, s.end)
        print(pretty_span(text, s.start, s.end, window=80))

    return pred2


In [ ]:
#Celda 3 — Pydantic: JSON estricto + offsets coherentes
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import Literal

Label = Literal["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]

class Span(BaseModel):
    label: Label
    start: int = Field(ge=0)
    end: int = Field(ge=0)
    quote: str = Field(min_length=1)  # substring literal para verificación

    @field_validator("end")
    @classmethod
    def end_after_start(cls, v, info):
        start = info.data.get("start")
        if start is not None and v <= start:
            raise ValueError("end must be > start")
        return v

class Pred(BaseModel):
    spans: List[Span]



In [ ]:
#Celda 4 — Helpers: normalización + verificación de offsets contra quote
def strict_verify(pred: Pred, text: str) -> Pred:
    ok_spans = []
    for s in pred.spans:
        if s.end > len(text):
            continue
        extracted = text[s.start:s.end]
        if extracted != s.quote:
            # Si no coincide EXACTO, descartamos (o lanza error si quieres)
            continue
        ok_spans.append(s)
    return Pred(spans=ok_spans)

def pretty_span(text: str, start: int, end: int, window: int = 80) -> str:
    a = max(0, start - window)
    b = min(len(text), end + window)
    ctx = text[a:b].replace("\n"," ")
    mark = text[start:end].replace("\n"," ")
    return f"...{ctx}...\n>>> [{start}:{end}] {mark}"


In [ ]:
#Celda 5 — Construcción de prompts Exp1/Exp2/Exp3
#Cambio crítico: pedimos quote además de start/end, y exigimos:
#quote debe ser exactamente text[start:end]
#start/end deben ser offsets del texto original que se da
LABELS = ["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]

SYSTEM_BASE = """Eres un anotador jurídico experto en contratos del sector público en España.
Tu tarea es identificar fragmentos textuales (segmentos/oraciones/cláusulas) que cumplen una función jurídica y devolver sus offsets start/end con una etiqueta.
Devuelves SOLO JSON válido.
Reglas:
- NO escribas explicaciones ni código.
Salida EXACTA: {"spans":[...]}.
- Si no encuentras ninguna etiqueta, devuelve {"spans": []}.
- PROHIBIDO devolver spans vacíos: nunca uses start=0,end=0 ni quote="". Si no hay evidencia, NO incluyas ese span.
- Cada span debe incluir: label, start, end, quote.
- quote debe ser EXACTAMENTE el substring literal del texto: quote == text[start:end].
- start/end son offsets 0-indexados sobre el texto ORIGINAL.
"""

def build_user_exp1(text: str) -> str:
    return f"""Tarea: detecta spans para estas etiquetas: {", ".join(LABELS)}.

Devuelve JSON:
{{
  "spans": [{{"label":"...", "start":0, "end":0, "quote":"..."}}]
}}

TEXTO:
\"\"\"{text}\"\"\"
"""

def memory_to_prompt(memory: List[Dict[str,Any]], uid_to_text: Dict[str,str], max_per_label: Optional[int]=None) -> str:
    # convierte memory_selected_FINAL en mini-few-shot con quote real
    by_label = {l: [] for l in LABELS}
    for ex in memory:
        by_label[ex["label"]].append(ex)

    parts = []
    for lab in LABELS:
        exs = by_label.get(lab, [])
        if max_per_label:
            exs = exs[:max_per_label]
        for ex in exs:
            txt = uid_to_text.get(ex["doc_uid"])
            if not txt:
                continue
            quote = txt[ex["start"]:ex["end"]].replace("\n"," ")
            parts.append(
                f"""Ejemplo ({lab})
Criterio: {ex["criterion"]}
Texto: \"\"\"{txt}\"\"\"
Respuesta: {{"spans":[{{"label":"{lab}","start":{ex["start"]},"end":{ex["end"]},"quote":"{quote}"}}]}}
"""
            )
    return "\n".join(parts)

def build_user_exp2(text: str, memory_block: str) -> str:
    return f"""Tarea: detecta spans para estas etiquetas: {", ".join(LABELS)}.

Aquí tienes ejemplos (memoria) con criterios y offsets correctos:
{memory_block}

Ahora anota el siguiente texto.

Devuelve JSON:
{{
  "spans": [{{"label":"...", "start":0, "end":0, "quote":"..."}}]
}}

TEXTO:
\"\"\"{text}\"\"\"
"""

def build_user_exp3(text: str, memory_block: str, fewshot_extra: str) -> str:
    return f"""Tarea: detecta spans para estas etiquetas: {", ".join(LABELS)}.

Memoria (curada):
{memory_block}

Few-shot adicional:
{fewshot_extra}

Devuelve JSON:
{{
  "spans": [{{"label":"...", "start":0, "end":0, "quote":"..."}}]
}}

TEXTO:
\"\"\"{text}\"\"\"
"""


In [ ]:
#Celda 6 — Preparar uid_to_text para construir memoria prompt
import hashlib

def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

uid_to_text = {}
for d in prompt_reg_docs:
    txt = d.get("text","")
    if txt:
        uid_to_text[stable_uid(txt)] = txt

# OJO: tu memory_selected_FINAL.json usa doc_uid de GOLD, no de prompt_regression.
# Para poder construir quotes, necesitamos cargar el GOLD también (mínimo texto por uid).
PATH_GOLD_JSON  = ROOT / "gold" / "corpus_annotated.json"
PATH_GOLD_JSONL = ROOT / "gold" / "corpus_annotated.jsonl"

def load_gold():
    if PATH_GOLD_JSON.exists():
        return json.loads(PATH_GOLD_JSON.read_text(encoding="utf-8"))
    if PATH_GOLD_JSONL.exists():
        return load_jsonl(PATH_GOLD_JSONL)
    raise FileNotFoundError("No encuentro corpus_annotated.json ni .jsonl")

gold = load_gold()
gold_uid_to_text = {}
for d in gold:
    txt = d.get("text","")
    if txt:
        gold_uid_to_text[stable_uid(txt)] = txt

print("gold docs:", len(gold_uid_to_text))


In [ ]:
#Celda 7 — Construir bloque de memoria para Exp2/Exp3
memory_block = memory_to_prompt(memory, gold_uid_to_text, max_per_label=None)
print("Memory block chars:", len(memory_block))
print(memory_block[:800])


## Para Few-shot extra para Exp3 (automático, sin leakage)

Idea: coger docs de prompt_regression que NO estén bloqueados y crear 1 ejemplo artificial por etiqueta usando spans GOLD…
Pero NB04 está corriendo sobre prompt_regression, que NO tiene tags. Para hacerlo “sin selección manual y sin leakage”, lo correcto es:
- construir few-shot extra desde GOLD (porque ahí sí hay tags),
- eligiendo docs no bloqueados, y
- tomando 1 span por etiqueta.

In [ ]:
#Celda 8 — Few-shot extra para Exp3 (automático, sin leakage)
def build_fewshot_extra_from_gold(gold: List[Dict[str,Any]], blocked_uids: set, k_per_label: int = 1):
    MVP = set(LABELS)
    picks = {lab: [] for lab in MVP}

    for d in gold:
        txt = d.get("text","")
        if not txt:
            continue
        uid = stable_uid(txt)
        if uid in blocked_uids:
            continue

        for t in d.get("tags", []):
            lab = t.get("tag")
            if lab not in MVP:
                continue
            if len(picks[lab]) >= k_per_label:
                continue
            s = int(t["start"]); e = int(t["end"])
            if 0 <= s < e <= len(txt):
                quote = txt[s:e].replace("\n"," ")
                picks[lab].append((txt, lab, s, e, quote))

        if all(len(picks[lab]) >= k_per_label for lab in MVP):
            break

    # Render en prompt
    blocks = []
    for lab in LABELS:
        for (txt, lab, s, e, quote) in picks[lab]:
            blocks.append(
                f"""Texto: \"\"\"{txt}\"\"\"
Respuesta: {{"spans":[{{"label":"{lab}","start":{s},"end":{e},"quote":"{quote}"}}]}}
"""
            )
    return "\n".join(blocks)

fewshot_extra = build_fewshot_extra_from_gold(gold, blocked_uids, k_per_label=1)
print("Few-shot extra chars:", len(fewshot_extra))
print(fewshot_extra[:1000])


In [ ]:
#Celda 9 — generate_chat() (la versión sin warnings)
import torch

def generate_chat(model, tokenizer, system: str, user: str,
                  max_new_tokens: int = 512,
                  temperature: float = 0.0,
                  top_p: float = 0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    if temperature and temperature > 0:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **gen_kwargs
        )

    gen_ids = out[0, input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [ ]:
#Celda 10 — Ejecutar 1 doc (supervisión real)
def run_one_exp(text: str, exp: int):
    if exp == 1:
        user = build_user_exp1(text)
    elif exp == 2:
        user = build_user_exp2(text, memory_block)
    elif exp == 3:
        user = build_user_exp3(text, memory_block, fewshot_extra)
    else:
        raise ValueError("exp debe ser 1,2,3")

    raw = generate_chat(model, tokenizer, SYSTEM_BASE, user, max_new_tokens=600, temperature=0.0)
    # intenta extraer JSON bruto (por si añade basura)
    m = re.search(r"\{.*\}\s*$", raw, flags=re.DOTALL)
    if m:
        raw = m.group(0)

    pred = Pred.model_validate_json(raw)
    pred2 = strict_verify(pred, text)  # descarta spans cuyo quote no coincide

    print("RAW JSON:", raw[:800], "...\n")
    print("Spans validados:", len(pred2.spans))
    for s in pred2.spans:
        print("\n", s.label, s.start, s.end)
        print(pretty_span(text, s.start, s.end, window=80))
    return pred2

sample = prompt_reg_docs[0]["text"]
_ = run_one_exp(sample, exp=1)


## Importante: Exp1 te va a dar muchísimos “spans: []” (es normal)

Exp1 (solo nombres de etiquetas) con un Llama 3.1 8B sin criterios suele:
- o inventar offsets,
- o devolverte vacío,
- o devolverte spans enormes.

Así que para depurar offsets “serios”, empieza por Exp2 (memoria+criterios), y luego Exp3.

`if exp == 1: user = build_user_exp1(text)`
      
`elif exp == 2: user = build_user_exp2(text, memory_block)`
       
`elif exp == 3: user = build_user_exp3(text, memory_block, fewshot_extra)`

- Exp1 suele ser flojo: etiquetas sin criterios → más alucinación.
- Exp2 debe mejorar: criterios + memoria.
- Exp3 añade estabilidad: few-shot extra desde GOLD no bloqueado.

In [ ]:
#probamos
sample = prompt_reg_docs[0]["text"]
_ = run_one_exp(sample, exp=2)


In [ ]:
#Celda 11 — Loop por documentos y guardado JSONL exp1/exp2/exp3. guardamos 3 ficheros jsonl en outputs/predictions
#Cada línea: {doc_uid, id(opcional), text_len, pred_spans:[...]}
def doc_uid(text: str) -> str:
    return stable_uid(text)

def predict_doc(d: Dict[str,Any], exp: int) -> Dict[str,Any]:
    text = d["text"]
    uid = doc_uid(text)

    if uid in blocked_uids:
        # NB04: para prompt_regression normalmente NO necesitas bloquear,
        # pero si quieres evitar ver textos “vistos” en memoria, lo dejamos:
        return {"doc_uid": uid, "skipped": True, "reason": "blocked_uid"}

    if exp == 1:
        user = build_user_exp1(text)
    elif exp == 2:
        user = build_user_exp2(text, memory_block)
    elif exp == 3:
        user = build_user_exp3(text, memory_block, fewshot_extra)
    else:
        raise ValueError("exp debe ser 1,2,3")

    raw = generate_chat(model, tokenizer, SYSTEM_BASE, user, max_new_tokens=600, temperature=0.0)

    # extraer JSON si el modelo mete texto extra
    m = re.search(r"\{.*\}\s*$", raw, flags=re.DOTALL)
    if m:
        raw = m.group(0)

    pred = Pred.model_validate_json(raw)
    pred = strict_verify(pred, text)

    return {
        "doc_uid": uid,
        "id": d.get("id"),
        "text_len": len(text),
        "spans": [s.model_dump() for s in pred.spans],
    }

def run_experiment(docs: List[Dict[str,Any]], exp: int, out_path: Path, limit: Optional[int]=None):
    n = 0
    with open(out_path, "w", encoding="utf-8") as f:
        for d in docs[: (limit or len(docs))]:
            try:
                rec = predict_doc(d, exp)
            except ValidationError as e:
                rec = {"doc_uid": doc_uid(d["text"]), "error": "pydantic_validation", "detail": str(e)}
            except Exception as e:
                rec = {"doc_uid": doc_uid(d["text"]), "error": "runtime", "detail": str(e)}

            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n += 1

    print(f"Exp{exp} guardado:", out_path, "| docs:", n)

OUT1 = OUT_DIR / "exp1.jsonl"
OUT2 = OUT_DIR / "exp2.jsonl"
OUT3 = OUT_DIR / "exp3.jsonl"

# primero prueba con 5 para depurar
run_experiment(prompt_reg_docs, exp=1, out_path=OUT1, limit=5)
run_experiment(prompt_reg_docs, exp=2, out_path=OUT2, limit=5)
run_experiment(prompt_reg_docs, exp=3, out_path=OUT3, limit=5)



In [ ]:
#Celda 12 — “Supervisor” rápido: inspeccionar predicciones guardadas
def read_jsonl(path: Path, n: int = 3):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= n:
                break
            rows.append(json.loads(line))
    return rows

rows = read_jsonl(OUT2, n=3)
for r in rows:
    print("\nDOC", r.get("doc_uid"), "spans:", len(r.get("spans",[])), "skipped:", r.get("skipped", False), "error:", r.get("error"))
    for s in r.get("spans", [])[:4]:
        print(" ", s["label"], s["start"], s["end"], "| quote[:60]=", s["quote"][:60])
